# Módulo 2: Scraping con Selenium
## LATAM Airlines
<a href="https://www.latam.com/es_ar/"><img src="https://i.pinimg.com/originals/dd/52/74/dd5274702d1382d696caeb6e0f6980c5.png"  width="420"></img></a>
<br>

Vamos a scrapear el sitio de Latam para averiguar datos de vuelos en funcion el origen y destino, fecha y cabina. La información que esperamos obtener de cada vuelo es:
- Precio(s) disponibles
- Horas de salida y llegada (duración)
- Información de las escalas

**¡Empecemos!**
Utilicemos lo aprendido hasta ahora para lograr el objetivo propuesto

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.latam.com/es_ar/apps/personas/booking?fecha1_dia=18&fecha1_anomes=2019-12&auAvailability=1&ida_vuelta=ida&vuelos_origen=Buenos%20Aires&from_city1=BUE&vuelos_destino=Madrid&to_city1=MAD&flex=1&vuelos_fecha_salida_ddmmaaaa=18/12/2019&cabina=Y&nadults=1&nchildren=0&ninfants=0&cod_promo=#/'

In [3]:
r = requests.get(url)

In [4]:
r.status_code

200

In [5]:
s = BeautifulSoup(r.text, 'lxml')

In [6]:
print(s.prettify())

<!DOCTYPE html>
<html lang="es">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <title>
   Selecciona tus vuelos | LATAM Airlines
  </title>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <link href="https://s.latamstatic.com/static/latam/images/favicon/LATAM/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
  <meta content="personas_bs3_latam_mu" name="layout"/>
  <meta content="LAN.com" name="description"/>
  <meta content="LAN.com" name="keywords"/>
  <meta content="B90D7B166BC8D2088504237E75BD91D8" name="sessionId" scheme="tracker"/>
  <meta content="latam" name="matchedScenario" scheme="tracker"/>
  <meta content="personas_bs3_latam_mu" name="template" scheme="tracker"/>
  <meta content="null" name="userDataCookieRawData" scheme="tracker"/>
  <meta content="3.0.53" name="appVersion" scheme="tracker"/>
  <style>
   .async-hide{opacity:0!important}
  

Vemos que la respuesta de la página no contiene la información que buscamos, ya que la misma aparece recién después de ejecutar el código JavaSCript que está en la respuesta.

## Selenium

Selenium es una herramienta que nos permitirá controlar un navegador y podremos utilizar las funcionalidades del motor de JavaScript para cargar el contenido que no viene en el HTML de la página. Para esto necesitamos el módulo `webdriver`.

In [7]:
from selenium import webdriver

Paso 1: instanciar un **driver** del navegador

In [8]:
options = webdriver.ChromeOptions()
options.add_argument('--incognito')
driver = webdriver.Chrome(executable_path='../chromedriver', options=options)

Paso 2: hacer que el navegador cargue la página web.

In [9]:
driver.get(url)

Paso 3: extraer la información de la página

In [11]:
vuelos = driver.find_elements_by_xpath('//li[@class="flight"]')
vuelos

[<selenium.webdriver.remote.webelement.WebElement (session="f08778eadd7976fafaf12a66aa2e1ce4", element="0ce97355-857d-4930-903a-9d362b361b42")>,
 <selenium.webdriver.remote.webelement.WebElement (session="f08778eadd7976fafaf12a66aa2e1ce4", element="e1b67b1b-516e-4c88-bf99-627e943abd7b")>,
 <selenium.webdriver.remote.webelement.WebElement (session="f08778eadd7976fafaf12a66aa2e1ce4", element="b74969f1-a15b-4f29-a330-c0980a4a69be")>,
 <selenium.webdriver.remote.webelement.WebElement (session="f08778eadd7976fafaf12a66aa2e1ce4", element="529cab8e-c38c-4b06-b692-5cbb92bbc810")>,
 <selenium.webdriver.remote.webelement.WebElement (session="f08778eadd7976fafaf12a66aa2e1ce4", element="be21f1e7-a549-43d6-a9be-200de6f22231")>,
 <selenium.webdriver.remote.webelement.WebElement (session="f08778eadd7976fafaf12a66aa2e1ce4", element="8e75084a-1fe6-4278-b9d3-10c14f76a856")>,
 <selenium.webdriver.remote.webelement.WebElement (session="f08778eadd7976fafaf12a66aa2e1ce4", element="306e13e6-2d36-4b05-ae08-a7

In [12]:
vuelo = vuelos[0]

In [13]:
vuelo

<selenium.webdriver.remote.webelement.WebElement (session="f08778eadd7976fafaf12a66aa2e1ce4", element="0ce97355-857d-4930-903a-9d362b361b42")>

In [14]:
# Hora de salida
vuelo.find_element_by_xpath('.//div[@class="departure"]/time').get_attribute('datetime')

'19:20'

In [15]:
# Hora de llegada
vuelo.find_element_by_xpath('.//div[@class="arrival"]/time').get_attribute('datetime')

'13:10'

In [16]:
# Duración del vuelo
vuelo.find_element_by_xpath('.//span[@class="duration"]/time').get_attribute('datetime')

'PT13H50M'

In [17]:
boton_escalas = vuelo.find_element_by_xpath('.//div[@class="flight-summary-stops-description"]/button')

In [18]:
boton_escalas.click()

In [19]:
segmentos = vuelo.find_elements_by_xpath('//div[@class="segments-graph"]/div[@class="segments-graph-segment"]')
segmentos

[<selenium.webdriver.remote.webelement.WebElement (session="f08778eadd7976fafaf12a66aa2e1ce4", element="5c21f956-1126-4837-9b2b-7845c8bf995d")>,
 <selenium.webdriver.remote.webelement.WebElement (session="f08778eadd7976fafaf12a66aa2e1ce4", element="85f411ae-60f0-4c34-9a7f-a5f4359b154c")>]

In [20]:
escalas = len(segmentos) - 1 

In [21]:
escalas

1

In [22]:
segmento = segmentos[0]

In [23]:
# Origen
segmento.find_element_by_xpath('.//div[@class="departure"]/span[@class="ground-point-name"]').text

'EZE'

In [24]:
# Hora de salida
segmento.find_element_by_xpath('.//div[@class="departure"]/time').get_attribute('datetime')

'19:20'

In [25]:
# Destino
segmento.find_element_by_xpath('.//div[@class="arrival"]/span[@class="ground-point-name"]').text

'GRU'

In [26]:
# Hora de llegada
segmento.find_element_by_xpath('.//div[@class="arrival"]/time').get_attribute('datetime')

'21:50'

In [27]:
# Duración del vuelo
segmento.find_element_by_xpath('.//span[@class="duration flight-schedule-duration"]/time').get_attribute('datetime')

'PT2H30M'

In [28]:
# Numero del vuelo
segmento.find_element_by_xpath('.//span[@class="equipment-airline-number"]').text

'LA8002'

In [29]:
# Modelo de avion
segmento.find_element_by_xpath('.//span[@class="equipment-airline-material"]').text

'Airbus 320-200'

In [30]:
# Duracion de la escala
segmento.find_element_by_xpath('.//div[@class="stop connection"]//p[@class="stop-wait-time"]//time').get_attribute('datetime')

'PT1H25M'

In [31]:
driver.find_element_by_xpath('//div[@class="modal-dialog"]//button[@class="close"]').click()

In [32]:
vuelo.click()

In [33]:
tarifas = vuelo.find_elements_by_xpath('.//div[@class="fares-table-container"]//tfoot//td[contains(@class, "fare-")]')

In [34]:
tarifas

[<selenium.webdriver.remote.webelement.WebElement (session="f08778eadd7976fafaf12a66aa2e1ce4", element="738ba2d4-11a9-4db3-8774-b25e3f871cd1")>,
 <selenium.webdriver.remote.webelement.WebElement (session="f08778eadd7976fafaf12a66aa2e1ce4", element="4a003cab-d056-4d69-95ae-c43304d46b99")>,
 <selenium.webdriver.remote.webelement.WebElement (session="f08778eadd7976fafaf12a66aa2e1ce4", element="f5b33902-71ee-4d3c-9f71-89f06c48fbca")>]

In [35]:
precios = []
for tarifa in tarifas:
    nombre = tarifa.find_element_by_xpath('.//label').get_attribute('for')
    moneda = tarifa.find_element_by_xpath('.//span[@class="price"]/span[@class="currency-symbol"]').text
    valor = tarifa.find_element_by_xpath('.//span[@class="price"]/span[@class="value"]').text 
    dict_tarifa={nombre:{'moneda':moneda, 'valor':valor}}
    precios.append(dict_tarifa)
    print(dict_tarifa)

{'LIGHT': {'moneda': 'US$', 'valor': '1282,40'}}
{'PLUS': {'moneda': 'US$', 'valor': '1335,90'}}
{'TOP': {'moneda': 'US$', 'valor': '1773,50'}}


In [46]:
def obtener_precios(vuelo):
    '''
    Función que retorna una lista de diccionarios con las distintas tarifas
    '''
    tarifas = vuelo.find_elements_by_xpath('.//div[@class="fares-table-container"]//tfoot//td[contains(@class, "fare-")]')
    precios = []
    for tarifa in tarifas:
        nombre = tarifa.find_element_by_xpath('.//label').get_attribute('for')
        moneda = tarifa.find_element_by_xpath('.//span[@class="price"]/span[@class="currency-symbol"]').text
        valor = tarifa.find_element_by_xpath('.//span[@class="price"]/span[@class="value"]').text 
        dict_tarifa={nombre:{'moneda':moneda, 'valor':valor}}
        precios.append(dict_tarifa)
    return precios

In [38]:
def obtener_datos_escalas(vuelo):
    '''
    Función que retorna una lista de diccionarios con la información de 
    las escalas de cada vuelo
    '''
    segmentos = vuelo.find_elements_by_xpath('//div[@class="segments-graph"]/div[@class="segments-graph-segment"]')
    info_escalas = []
    for segmento in segmentos:
        # Origen
        origen = segmento.find_element_by_xpath('.//div[@class="departure"]/span[@class="ground-point-name"]').text
        # Hora de salida
        dep_time = segmento.find_element_by_xpath('.//div[@class="departure"]/time').get_attribute('datetime')
        # Destino
        destino = segmento.find_element_by_xpath('.//div[@class="arrival"]/span[@class="ground-point-name"]').text
        # Hora de llegada
        arr_time = segmento.find_element_by_xpath('.//div[@class="arrival"]/time').get_attribute('datetime')
        # Duración del vuelo
        duracion_vuelo = segmento.find_element_by_xpath('.//span[@class="duration flight-schedule-duration"]/time').get_attribute('datetime')
        # Numero del vuelo
        numero_vuelo = segmento.find_element_by_xpath('.//span[@class="equipment-airline-number"]').text
        # Modelo de avion
        modelo_avion =segmento.find_element_by_xpath('.//span[@class="equipment-airline-material"]').text
        if segmento != segmentos[-1]:
            # Duracion de la escala
            duracion_escala = segmento.find_element_by_xpath('.//div[@class="stop connection"]//p[@class="stop-wait-time"]//time').get_attribute('datetime')
        else:
            duracion_escala = ''
        
        data_dict={
            'origen':origen,
            'dep_time':dep_time,
            'destino':destino,
            'arr_time':arr_time,
            'duracion_vuelo':duracion_vuelo,
            'numero_vuelo':numero_vuelo,
            'modelo_avion':modelo_avion,
            'duracion_escala':duracion_escala,
        }
        info_escalas.append(data_dict)
    return info_escalas

In [41]:
def obtener_tiempos(vuelo):
    '''
    Función que retorna un diccionario con los horarios de salida y llegada de cada
    vuelo, incluyendo la duración. 
    Nota: la duración del vuelo no es la hora de llegada - la hora de salida porque
    puede haber diferencia de horarios entre el origen y el destino.
    '''
    # Hora de salida
    salida = vuelo.find_element_by_xpath('.//div[@class="departure"]/time').get_attribute('datetime')
    # Hora de llegada
    llegada = vuelo.find_element_by_xpath('.//div[@class="arrival"]/time').get_attribute('datetime')
    # Duracion
    duracion = vuelo.find_element_by_xpath('.//span[@class="duration"]/time').get_attribute('datetime')
    tiempos = {'hora_salida': salida, 'hora_llegada': llegada, 'duracion': duracion}
    return tiempos


In [64]:
def obtener_info(driver):
    vuelos = driver.find_elements_by_xpath('//li[@class="flight"]')
    print(f'Se encontraron {len(vuelos)} vuelos.')
    print('Iniciando scraping...')
    info = []
    for vuelo in vuelos:
        #obtenemos los tiempos generales de cada vuelo
        tiempos = obtener_tiempos(vuelo)
        # Clickeamos sobre el boton de las escalas
        vuelo.find_element_by_xpath('.//div[@class="flight-summary-stops-description"]/button').click()
        escalas = obtener_datos_escalas(vuelo)
        # Cerramos el modal
        driver.find_element_by_xpath('//div[@class="modal-dialog"]//button[@class="close"]').click()
        # Clickeamos el vuelo para ver los precios
        vuelo.click()
        precios = obtener_precios(vuelo)
        vuelo.click()
        info.append({'precios':precios, 'tiempos': tiempos, 'escalas':escalas})
    return info

In [71]:
import time

In [73]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

In [74]:
options = webdriver.ChromeOptions()
options.add_argument('--incognito')
driver = webdriver.Chrome(executable_path='../chromedriver', options=options)
driver.get(url)
# Introducir una demora
delay = 10
try:
    # introducir demora inteligente
    vuelo = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//li[@class="flight"]')))
    print('La página terminó de cargar')
    info_vuelos = obtener_info(driver)
except TimeoutException:
    print('La página tardó demasiado en cargar')
    info_vuelos = []
driver.close()

La página terminó de cargar
Se encontraron 12 vuelos.
Iniciando scraping...


In [67]:
info_vuelos

Se encontraron 12 vuelos.
Iniciando scraping...


[{'precios': [{'LIGHT': {'moneda': '', 'valor': '1282,40'}},
   {'PLUS': {'moneda': 'US$', 'valor': '1335,90'}},
   {'TOP': {'moneda': 'US$', 'valor': '1773,50'}}],
  'tiempos': {'hora_salida': '19:20',
   'hora_llegada': '13:10',
   'duracion': 'PT13H50M'},
  'escalas': [{'origen': 'EZE',
    'dep_time': '19:20',
    'destino': 'GRU',
    'arr_time': '21:50',
    'duracion_vuelo': 'PT2H30M',
    'numero_vuelo': 'LA8002',
    'modelo_avion': 'Airbus 320-200',
    'duracion_escala': 'PT1H25M'},
   {'origen': 'GRU',
    'dep_time': '23:15',
    'destino': 'MAD',
    'arr_time': '13:10',
    'duracion_vuelo': 'PT9H55M',
    'numero_vuelo': 'LA8064',
    'modelo_avion': 'Airbus 350-900',
    'duracion_escala': ''}]},
 {'precios': [{'LIGHT': {'moneda': 'US$', 'valor': '1282,40'}},
   {'PLUS': {'moneda': 'US$', 'valor': '1335,90'}},
   {'TOP': {'moneda': 'US$', 'valor': '1773,50'}}],
  'tiempos': {'hora_salida': '16:50',
   'hora_llegada': '13:10',
   'duracion': 'PT16H20M'},
  'escalas': [{

Paso 4: cerrar el navegador

In [70]:
driver.close()